<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Homework/4_Classification/PH_5_Bayes_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3 
Apply our two new models (multinomial naive Bayes and a neural network) to the 20-Newsgroups dataset.

In [3]:
# imports
import re
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import train_test_split
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [4]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load datasets

In [5]:
#Load stemmed data as .pkl
with open('/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Newsgroup/data_stemmed.pkl', 'rb') as f:
    data_stemmed_list = pickle.load(f)

stemmed_df = pd.DataFrame(data_stemmed_list, columns=["stemmed_text"])

In [6]:
#Load lemmatized data as .pkl
with open('/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Newsgroup/data_lemma.pkl', 'rb') as f:
    data_lemma_list = pickle.load(f)

lemma_df = pd.DataFrame(data_lemma_list, columns=["stemmed_text"])

In [7]:
# load y data
y_df = pd.read_json("https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json")[["target","target_names"]]
y_df.head()

,target,target_names
0,7,rec.autos
1,4,comp.sys.mac.hardware
2,4,comp.sys.mac.hardware
3,1,comp.graphics
4,14,sci.space


In [8]:
stemmed_y_df = pd.concat([stemmed_df, y_df], axis=1)
lemma_y_df = pd.concat([lemma_df, y_df], axis=1)

In [9]:
lemma_y_df.columns

Index(['stemmed_text', 'target', 'target_names'], dtype='object')

In [10]:
# train test split
X_stem = stemmed_y_df[["stemmed_text"]]
y_stem = stemmed_y_df['target']

X_s_train, X_s_test, y_s_train, y_s_test = train_test_split(X_stem, y_stem, test_size=0.1)

# train test split
X_lem = lemma_y_df[["stemmed_text"]]
y_lem = lemma_y_df['target']

X_l_train, X_l_test, y_l_train, y_l_test = train_test_split(X_lem, y_lem, test_size=0.1)

## Multinomial Naive Bayes

In [11]:
# Instantiate Vectorizers 

#Stemmed
abs_vec_s = CountVectorizer(max_df=0.95, min_df=0.01)
rel_vec_s = TfidfVectorizer(max_df=0.95, min_df=0.01, use_idf=False, norm='l1') # use-idf: Enable inverse-document-frequency reweighting; norm: Each output row will have unit norm (l1 = Sum of absolute values of vector elements is 1.)
idf_vec_s = TfidfVectorizer(max_df=0.95, min_df=0.01, smooth_idf=True) # smooth_idf: Smooth idf weights by adding one to document frequencies (Prevents zero divisions) -> not regularized

#Lemma
abs_vec_l = CountVectorizer(max_df=0.95, min_df=0.01)
rel_vec_l = TfidfVectorizer(max_df=0.95, min_df=0.01, use_idf=False, norm='l1') # use-idf: Enable inverse-document-frequency reweighting; norm: Each output row will have unit norm (l1 = Sum of absolute values of vector elements is 1.)
idf_vec_l = TfidfVectorizer(max_df=0.95, min_df=0.01, smooth_idf=True) # smooth_idf: Smooth idf weights by adding one to document frequencies (Prevents zero divisions) -> not regularized

### Abs. Freq. 

#### Stemmed

In [12]:
X_train_abs_stem = abs_vec_s.fit_transform(X_s_train["stemmed_text"]).toarray()
X_test_abs_stem = abs_vec_s.transform(X_s_test["stemmed_text"]).toarray()

In [13]:
clf_s = MultinomialNB()
clf_s.fit(X_train_abs_stem, y_s_train)

MultinomialNB()

In [14]:
y_pred_s = clf_s.predict(X_test_abs_stem)
print('Train accuracy: ', clf_s.score(X_train_abs_stem, y_s_train))
print('Test accuracy: ', accuracy_score(y_pred_s, y_s_test))

Train accuracy:  0.8475741504615989
Test accuracy:  0.7756183745583038


#### Lemmatized

In [15]:
X_train_abs_lem = abs_vec_l.fit_transform(X_l_train["stemmed_text"]).toarray()
X_test_abs_lem = abs_vec_l.transform(X_l_test["stemmed_text"]).toarray()

In [16]:
clf_l = MultinomialNB()
clf_l.fit(X_train_abs_lem, y_l_train)

MultinomialNB()

In [17]:
y_pred_l = clf_l.predict(X_test_abs_lem)
print('Train accuracy: ', clf_l.score(X_train_abs_lem, y_l_train))
print('Test accuracy: ', accuracy_score(y_pred_l, y_l_test))

Train accuracy:  0.838243959929287
Test accuracy:  0.7791519434628975


### Rel. Freq. 

#### Stemmed

In [18]:
X_train_rel_stem = rel_vec_s.fit_transform(X_s_train["stemmed_text"]).toarray()
X_test_rel_stem = rel_vec_s.transform(X_s_test["stemmed_text"]).toarray()

In [19]:
clf_s = MultinomialNB()
clf_s.fit(X_train_rel_stem, y_s_train)

MultinomialNB()

In [20]:
y_pred_s = clf_s.predict(X_test_rel_stem)
print('Train accuracy: ', clf_s.score(X_train_rel_stem, y_s_train))
print('Test accuracy: ', accuracy_score(y_pred_s, y_s_test))

Train accuracy:  0.7423885287762718
Test accuracy:  0.6881625441696113


#### Lemmatized

In [21]:
X_train_rel_lem = rel_vec_l.fit_transform(X_l_train["stemmed_text"]).toarray()
X_test_rel_lem = rel_vec_l.transform(X_l_test["stemmed_text"]).toarray()

In [22]:
clf_l = MultinomialNB()
clf_l.fit(X_train_rel_lem, y_l_train)

MultinomialNB()

In [23]:
y_pred_l = clf_l.predict(X_test_rel_lem)
print('Train accuracy: ', clf_l.score(X_train_rel_lem, y_l_train))
print('Test accuracy: ', accuracy_score(y_pred_l, y_l_test))

Train accuracy:  0.7229424474562954
Test accuracy:  0.6607773851590106


### TF-Idf

#### Stemmed

In [24]:
X_train_idf_stem = idf_vec_s.fit_transform(X_s_train["stemmed_text"]).toarray()
X_test_idf_stem = idf_vec_s.transform(X_s_test["stemmed_text"]).toarray()

In [25]:
clf_s = MultinomialNB()
clf_s.fit(X_train_idf_stem, y_s_train)

MultinomialNB()

In [26]:
y_pred_s = clf_s.predict(X_test_idf_stem)
print('Train accuracy: ', clf_s.score(X_train_idf_stem, y_s_train))
print('Test accuracy: ', accuracy_score(y_pred_s, y_s_test))

Train accuracy:  0.8622078177175407
Test accuracy:  0.7959363957597173


#### Lemmatized

In [27]:
X_train_idf_lem = idf_vec_l.fit_transform(X_l_train["stemmed_text"]).toarray()
X_test_idf_lem = idf_vec_l.transform(X_l_test["stemmed_text"]).toarray()

In [28]:
clf_l = MultinomialNB()
clf_l.fit(X_train_idf_lem, y_l_train)

MultinomialNB()

In [29]:
y_pred_l = clf_l.predict(X_test_idf_lem)
print('Train accuracy: ', clf_l.score(X_train_idf_lem, y_l_train))
print('Test accuracy: ', accuracy_score(y_pred_l, y_l_test))

Train accuracy:  0.8516990768021999
Test accuracy:  0.7959363957597173


## Neural Networks

In [92]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping

In [40]:
# no of classes in dataset
y_df.target.nunique()

20

In [107]:
from tensorflow.keras.utils import to_categorical
y_l_train_cat = to_categorical(y_l_train)
y_l_test_cat = to_categorical(y_l_test)

In [74]:
#input dim
X_train_abs_lem.shape

(10182, 1566)

In [121]:
# Classification with 20 classes
model = Sequential()
model.add(layers.Dense(30, activation='relu', input_dim=1566))
#model.add(layers.Dense(20, activation='relu'))
#model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

In [122]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics = ['accuracy'])

In [123]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 30)                47010     
                                                                 
 dense_33 (Dense)            (None, 20)                620       
                                                                 
Total params: 47,630
Trainable params: 47,630
Non-trainable params: 0
_________________________________________________________________


In [124]:
#Early stopping
es = EarlyStopping(patience=20, restore_best_weights=True)

In [125]:
model.fit(np.asarray(X_train_abs_lem).astype("float32"), y_l_train_cat, epochs=100, validation_split=0.3, callbacks=[es])

Epoch 1/100
223/223 [==============================] - 2s 4ms/step - loss: 2.2641 - accuracy: 0.4571 - val_loss: 1.5320 - val_accuracy: 0.6995
Epoch 2/100
223/223 [==============================] - 1s 3ms/step - loss: 1.1226 - accuracy: 0.7929 - val_loss: 1.0453 - val_accuracy: 0.7620
Epoch 3/100
223/223 [==============================] - 1s 3ms/step - loss: 0.7336 - accuracy: 0.8548 - val_loss: 0.8742 - val_accuracy: 0.7869
Epoch 4/100
223/223 [==============================] - 1s 3ms/step - loss: 0.5393 - accuracy: 0.8914 - val_loss: 0.7993 - val_accuracy: 0.7931
Epoch 5/100
223/223 [==============================] - 1s 3ms/step - loss: 0.4183 - accuracy: 0.9176 - val_loss: 0.7694 - val_accuracy: 0.8003
Epoch 6/100
223/223 [==============================] - 1s 3ms/step - loss: 0.3339 - accuracy: 0.9364 - val_loss: 0.7604 - val_accuracy: 0.8049
Epoch 7/100
223/223 [==============================] - 1s 5ms/step - loss: 0.2696 - accuracy: 0.9530 - val_loss: 0.7613 - val_accuracy: 0.7993

In [126]:
results = model.evaluate(np.asarray(X_test_abs_lem).astype("float32"), y_l_test_cat, verbose=0)
print(results[1])

0.8038869500160217
